In [2]:
import json
import requests # to download some resources
import os # file operations
import numpy as np # linear algebra
import pandas as pd # data processing

### Combining all countries - overview

In [20]:
# Consolidating answers from the whole sample
# Folder Path
testing_json = f"../data/4-summary-responses-json/version_yesno_updated/Testing_Sample"
design_json = f"../data/4-summary-responses-json/version_yesno_updated/Design_Sample"
full_json = f"../data/4-summary-responses-json/version_yesno_updated/Testing_FullSample"

# Lista para almacenar los DataFrames
dfs = []

# Recorrer todos los archivos en la carpeta
for file_name in os.listdir(design_json):
    if file_name.endswith(".json"): 
        file_path = os.path.join(design_json, file_name) 
        
        df = pd.read_json(file_path)
        dfs.append(df)

for file_name in os.listdir(testing_json):
    if file_name.endswith(".json"): 
        file_path = os.path.join(testing_json, file_name) 
    
        df = pd.read_json(file_path)
        dfs.append(df)

for file_name in os.listdir(full_json):
    if file_name.endswith(".json"): 
        file_path = os.path.join(full_json, file_name) 
        
        df = pd.read_json(file_path) 
        dfs.append(df)

# Concatenar todos los DataFrames en uno solo
combined_data = pd.concat(dfs, ignore_index=True)

# Collapse by country and question
combined_data = combined_data.groupby(['country', 'question'], as_index=False).agg({
    'answer': lambda x: ', '.join(x.dropna().astype(str).tolist()),
    'supporting_chunks': 'first',
    'answer_full': 'first',
    'supporting_text': 'first'
})

combined_data['question'] = combined_data['question'].astype(str)
combined_data['country'] = combined_data['country'].astype(str)
combined_data = combined_data.rename(columns={"answer": "answer_chunking"})

combined_data = combined_data.drop(columns=['supporting_chunks', 'supporting_text'])

# Keep one answer, answer_full preferred over answer_chunking. If answer_full is missing, use answer_chunking
combined_data['answer'] = combined_data['answer_full']
combined_data.loc[combined_data['answer'].isnull(), 'answer'] = combined_data['answer_chunking']

full_data_countries = combined_data[['country', 'question', 'answer']]

# Save the data
output_path = f"../data/6-performance/combined_all_countries_overview.xlsx"  # Nombre del archivo Excel
full_data_countries.to_excel(output_path, index=False)

In [21]:
full_data_countries

,country,question,answer
0,afghanistan,1,yes
1,afghanistan,2,yes
2,afghanistan,3,yes
3,afghanistan,4,yes
4,afghanistan,5,yes
...,...,...,...
1579,zambia,20,yes
1580,zambia,21,yes
1581,zambia,22,yes
1582,zambia,period_end,2025
